In [1]:
import numpy as np
import os
import scipy.io
from sklearn.metrics import classification_report,confusion_matrix

In [2]:
#Please add the folder name of the dataset to run it on different dataset.
dataset = 'CUB'
path = 'E:/Sushree/Dataset/data/xlsa17/data/'

res101 = scipy.io.loadmat(path + dataset + '/res101.mat')
att_splits = scipy.io.loadmat(path + dataset + '/att_splits.mat')

In [3]:
signature = att_splits['att']
#signature = att_splits['original_att']
#signature = signature/100
print(signature.shape) #(312, 200)

attribute = signature.transpose()
attribute[attribute<0] = 0
print(attribute, attribute.shape)#(200, 312)

(312, 200)
[[0.0106384  0.0106384  0.00709227 ... 0.00918617 0.02526198 0.02066889]
 [0.         0.01133243 0.00944369 ... 0.00266542 0.02132333 0.05863916]
 [0.         0.         0.00742474 ... 0.         0.00885258 0.01770516]
 ...
 [0.         0.00334966 0.         ... 0.00556558 0.         0.15027069]
 [0.         0.11184146 0.         ... 0.08207164 0.05836206 0.01823814]
 [0.04378019 0.02814441 0.         ... 0.06022509 0.07695428 0.06189801]] (200, 312)


In [4]:
import gensim.downloader as api
import pandas as pd
print('Load pretrain w2v model')

#model_name = 'word2vec-google-news-300' # length = 300
#model = api.load(model_name)

import transformers
from transformers import BertTokenizer, TFBertModel, BartTokenizer, TFBartModel, GPT2Tokenizer, TFGPT2Model
from transformers import OpenAIGPTTokenizer, TFOpenAIGPTModel

# Bert model
#---------------------------------------------------------------------------
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # length 768
model = TFBertModel.from_pretrained("bert-base-uncased")

#tokenizer = BertTokenizer.from_pretrained('bert-large-uncased') # 1024
#model = TFBertModel.from_pretrained("bert-large-uncased")

# Bart model
#---------------------------------------------------------------------------
#tokenizer = BartTokenizer.from_pretrained('facebook/bart-base') # length 768
#model = TFBartModel.from_pretrained("facebook/bart-base")

#tokenizer = BartTokenizer.from_pretrained('facebook/bart-large') # length 1024
#model = TFBartModel.from_pretrained("facebook/bart-large")

# GPT2 model
#---------------------------------------------------------------------------
#tokenizer = GPT2Tokenizer.from_pretrained('gpt2') # length 768
#model = TFGPT2Model.from_pretrained('gpt2')

# GPT model
#---------------------------------------------------------------------------
#tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')# length 768
#model = TFOpenAIGPTModel.from_pretrained('openai-gpt')

Load pretrain w2v model


C:\Users\Admin\anaconda3\envs\en3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassifica

In [5]:
#tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')
#model = TFOpenAIGPTModel.from_pretrained('openai-gpt')
#encoded_input = tokenizer('sushree', return_tensors='tf')
#print(encoded_input)
#length = encoded_input.input_ids.shape[1]
#print(length)

In [6]:
#print(tokenizer)
#out = model(encoded_input).last_hidden_state[:, 2, :]
#print(out.shape)

In [7]:

dim_w2v = 768

#%%
#For CUB
replace_word = [('spatulate','broad'),('upperparts','upper parts'),('grey','gray'), ('eyering', 'eye ring')] # for CUB


path = 'E:/Sushree/Dataset/CUB_200_2011/CUB_200_2011/attributes/attributes.txt'
df=pd.read_csv(path,sep=' ',header = None, names = ['idx','des'])
des = df['des'].values

#%% replace out of dictionary (OOD) words
for pair in replace_word:
    for idx,s in enumerate(des):
        des[idx] = s.replace(pair[0],pair[1])
print('Done replacing OOD words')

#%% filter
new_des = [' '.join(i.split('_')) for i in des]
new_des = [' '.join(i.split('-')) for i in new_des]
new_des = [' '.join(i.split('::')) for i in new_des]
new_des = [i.split('(')[0] for i in new_des]
new_des = [i[4:] for i in new_des]

df['new_des'] = des
df.to_csv('E:/Sushree/Dataset/CUB_200_2011/CUB_200_2011/attributes/new_des.csv')
print('Done preprocessing attribute des')
import pickle


# for word2vec
# -----------------------------------------------------------------------------------
#counter_err = 0
#
#all_w2v = []
#for s in new_des:
#    print(s)
#    words = s.split(' ')
#    if words[-1] == '':     #remove empty element
#        words = words[:-1]
#    w2v = np.zeros(dim_w2v)
#    for w in words:
#        try:
#            w2v += model[w]
#        except Exception as e:
#            print(e)
#            counter_err += 1
#    w2v = w2v / len(words)  
#    all_w2v.append(w2v[np.newaxis,:])
    
#print('counter_err ',counter_err)

#w2v_att = np.concatenate(all_w2v,axis=0)
#print(w2v_att, w2v_att.shape)

# for Bert attributes
# -----------------------------------------------------------------------------------
counter_err = 0
counter = 0
w2v_att = np.zeros((signature.shape[0], dim_w2v))
for s in des:
    print(s)
    w2v = np.zeros(dim_w2v)
    encoded_input = tokenizer(s, return_tensors='tf')
    length = encoded_input.input_ids.shape[1]
    #print(length)
    for i in range(length-2): # for Bert, Bart
    #for i in range(length): # for GPT2
        try:
            w2v = w2v + model(encoded_input).last_hidden_state[:, i+1, :] # for Bert, Bart
            #w2v = w2v + model(encoded_input).last_hidden_state[:, i, :] # for GPT2
        #print(model(encoded_input).last_hidden_state[:, i+1, :][:,1:2])
        #print(w2v[:, 1:2])
        #print(w2v.shape)
        except Exception as e:
            print(e)
            counter_err += 1
    w2v = w2v / (length - 2)# for Bert, Bart
    #w2v = w2v / length # for GPT2
    #print(w2v[:, 1:2])
    w2v_att[counter] = w2v
    counter = counter + 1

print('counter_err ',counter_err)
print(w2v_att, w2v_att.shape)
# -----------------------------------------------------------------------------------


attribute_new = np.einsum('ij,jl->ijl', attribute, w2v_att)

attribute_new = np.reshape(attribute_new, [attribute_new.shape[0], attribute_new.shape[1]* attribute_new.shape[2]])
print(attribute_new, attribute_new.shape)

#attribute_new[attribute_new<0] = 0
#print(attribute_new, attribute_new.shape)

Done replacing OOD words
Done preprocessing attribute des
has_bill_shape::curved_(up_or_down)
has_bill_shape::dagger
has_bill_shape::hooked
has_bill_shape::needle
has_bill_shape::hooked_seabird
has_bill_shape::broad
has_bill_shape::all-purpose
has_bill_shape::cone
has_bill_shape::specialized
has_wing_color::blue
has_wing_color::brown
has_wing_color::iridescent
has_wing_color::purple
has_wing_color::rufous
has_wing_color::gray
has_wing_color::yellow
has_wing_color::olive
has_wing_color::green
has_wing_color::pink
has_wing_color::orange
has_wing_color::black
has_wing_color::white
has_wing_color::red
has_wing_color::buff
has_upper parts_color::blue
has_upper parts_color::brown
has_upper parts_color::iridescent
has_upper parts_color::purple
has_upper parts_color::rufous
has_upper parts_color::gray
has_upper parts_color::yellow
has_upper parts_color::olive
has_upper parts_color::green
has_upper parts_color::pink
has_upper parts_color::orange
has_upper parts_color::black
has_upper parts_colo

In [8]:

# total number of instances or images = 11788: ranges from 0 to 11787

trainval_loc = np.squeeze(att_splits['trainval_loc']-1) # -1: to consider the overflow problem
print(np.unique(trainval_loc), np.max(np.unique(trainval_loc))) # smallest location: 2, largest location 11787

labels = res101['labels']# direct class labels
print('labels', labels, labels.shape)# 11788 x 1

print('unique_labels', np.unique(labels), np.unique(labels).shape)# class labels range from 1 to 200, 200 classes

# get the labels for trainval, test seen and test unseen sets

labels_trainval = labels[trainval_loc]
print('labels_trainval', labels_trainval, labels_trainval.shape)

unique_labels_trainval = np.unique(labels_trainval) # labels min:1 max:200
print('unique_labels_trainval', unique_labels_trainval, unique_labels_trainval.shape)# 200 classes



[    1     2     4 ... 11724 11725 11726] 11726
labels [[151]
 [151]
 [151]
 ...
 [150]
 [150]
 [150]] (11788, 1)
unique_labels [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200] (200,)
labels_trainval [[197]
 [198]
 [ 31]
 ...
 [ 65]
 [1

In [9]:
X_features = res101['features']

# locations are already subtracted by 1, so they range from 0 to 11787
trainval_vec = X_features[:, trainval_loc].transpose()
print("Features for trainval:", trainval_vec.shape) #(7057, 2048)

# attribute is defined for all 200 classes, so we cant use locations directly, instead we have to use labels 
# that range from 1 to 200, so we have to subtract 1

trainval_attributes = np.zeros((len(trainval_loc), attribute_new.shape[1]))
for i in range(len(trainval_loc)):
    trainval_attributes[i] = attribute_new[int(labels_trainval[i])-1]

print(trainval_attributes, trainval_attributes.shape)# (7057, 319488)


Features for trainval: (7057, 2048)
[[ 1.76200075e-04  2.54372776e-05  1.68130403e-03 ... -1.62484619e-02
  -2.15702232e-02  7.03285847e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -1.59936223e-02
  -2.12319176e-02  6.92255570e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -1.31450454e-02
  -1.74503634e-02  5.68959972e-03]
 ...
 [ 5.50317675e-04  7.94470917e-05  5.25114036e-03 ... -1.58951398e-02
  -2.11011797e-02  6.87992929e-03]
 [ 3.32486331e-04  4.79996795e-05  3.17259008e-03 ... -1.64188769e-03
  -2.17964532e-03  7.10661960e-04]
 [ 2.44718696e-04  3.53290283e-05  2.33510985e-03 ... -1.48075870e-03
  -1.96574272e-03  6.40920134e-04]] (7057, 239616)


In [10]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from keras.optimizers import SGD, Adam, Adagrad
import keras.backend as K

attribute_shape_new = attribute_new.shape[1]
print(attribute_shape_new)

# define model for attribute transformation

inputt = Input(shape = attribute_shape_new)
hidden = Dense(4096, name="layer1", activation='linear')(inputt)
output = Dense(2048, name="layer3", activation='linear')(hidden)

model_transform_attribute = Model(inputs = inputt, outputs = output)

#sgd = SGD(learning_rate = 1e-2, decay = 1e-6, momentum = 0.9, nesterov = True)
opt = Adam(learning_rate = 1e-2, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0001)

model_transform_attribute.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

for layer in model_transform_attribute.layers:
        layer.trainable = False
        
model_transform_attribute.summary()

attribute_new = model_transform_attribute.predict(attribute_new)
print(attribute_new, attribute_new.shape)


trainval_attributes = np.zeros((len(trainval_loc), attribute_new.shape[1]))
for i in range(len(trainval_loc)):
    trainval_attributes[i] = attribute_new[int(labels_trainval[i])-1]

print(trainval_attributes, trainval_attributes.shape)# (7057, 85)

239616
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 239616)]          0         
                                                                 
 layer1 (Dense)              (None, 4096)              981471232 
                                                                 
 layer3 (Dense)              (None, 2048)              8390656   
                                                                 
Total params: 989,861,888
Trainable params: 0
Non-trainable params: 989,861,888
_________________________________________________________________
7/7 [==============================] - 0s 17ms/step
[[-0.00659391  0.01807893  0.04017409 ... -0.0203063  -0.01138855
   0.00085992]
 [-0.00271799 -0.01195218 -0.00141738 ... -0.03540748  0.01226223
   0.00674735]
 [-0.00342696 -0.00230486 -0.01704866 ... -0.00140131  0.02639913
  -0.04940924]
 ...
 [ 0

In [11]:
trainval_input1 = np.reshape(trainval_vec, [trainval_vec.shape[0], 1, 1, trainval_vec.shape[1]])
print(trainval_input1.shape)

trainval_input2 = np.reshape(trainval_attributes, [trainval_attributes.shape[0], 1, 1, trainval_attributes.shape[1]])
print(trainval_input2.shape)

(7057, 1, 1, 2048)
(7057, 1, 1, 2048)


In [12]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence

X_train, X_val, y_train, y_val = train_test_split(trainval_input1, trainval_input2, test_size = 0.2, random_state = 42)

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y
    
batch_size = 8
train_gen = DataGenerator(X_train, y_train, batch_size)   
val_gen = DataGenerator(X_val, y_val, batch_size)

In [13]:
# embedding model
# -----------------------------------------------------------------------------------
inputt = Input(shape = (1, 1, trainval_vec.shape[1]))
hidden1 = Conv2D(2048, 1, name="layer1", activation='relu')(inputt)
output1 = Conv2D(1024, 1, name="layer2", activation='relu')(hidden1)
hidden2 = Conv2D(1024, 1, name="layer3", activation='relu')(output1)
output2 = Conv2D(trainval_attributes.shape[1], 1, name="layer4", activation='linear')(hidden2)

embedding_model = Model(inputs = inputt, outputs = output2)

# embedding model variants
# -----------------------------------------------------------------------------------

#inputt = Input(shape = (1, 1, trainval_vec.shape[1]))
#hidden1 = Conv2D(1024, 1, name="layer1", activation='relu')(inputt)
#output1 = Conv2D(1024, 1, name="layer2", activation='relu')(hidden1)
#hidden2 = Conv2D(1024, 1, name="layer3", activation='relu')(output1)
#output2 = Conv2D(trainval_attributes.shape[1], 1, name="layer4", activation='linear')(hidden2)

#embedding_model = Model(inputs = inputt, outputs = output2)


adam = Adam(learning_rate = 0.0001, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0001)
embedding_model.compile(adam, loss = tf.keras.losses.CosineSimilarity(axis=-1, reduction=tf.keras.losses.Reduction.AUTO), metrics = ['accuracy'])
#embedding_model.compile(adam, loss = cosine_loss, metrics = ['accuracy'])

embedding_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1, 1, 2048)]      0         
                                                                 
 layer1 (Conv2D)             (None, 1, 1, 2048)        4196352   
                                                                 
 layer2 (Conv2D)             (None, 1, 1, 1024)        2098176   
                                                                 
 layer3 (Conv2D)             (None, 1, 1, 1024)        1049600   
                                                                 
 layer4 (Conv2D)             (None, 1, 1, 2048)        2099200   
                                                                 
Total params: 9,443,328
Trainable params: 9,443,328
Non-trainable params: 0
_________________________________________________________________


In [14]:

#CosineSimilarity is a number between -1 and 1. When it is a negative number between -1 and 0, 0 indicates orthogonality and 
#values closer to -1 indicate greater similarity. The values closer to 1 indicate greater dissimilarity. 

save_path = 'C:/Users/Admin/Sushree_Codes/Sush_3/Results/CUB/BertAttribute/with_LE/Embedding Model/'
name = 'Reembedding_model_Bert_2048_1024_1024_CUB_200eph_adam_cos_16bch_0.0001lr'


#file_path = save_path + 'bw_' + name + '.h5'

#model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#    filepath = file_path,
#    monitor = 'val_accuracy',
#    mode = 'max',
#    save_best_only=True)

#train_summary = embedding_model.fit(train_gen, epochs = 200, verbose = 1, callbacks = [model_checkpoint_callback], validation_data = val_gen, 
#                              shuffle = True, steps_per_epoch = len(train_gen)//batch_size, 
#                              validation_steps = len(val_gen)//batch_size)

embedding_model.load_weights(save_path + 'bw_' + name + '.h5')

In [15]:
embedding_model_0 = Model(inputs = embedding_model.input, outputs = embedding_model.layers[-3].output) # for embedding model
#embedding_model_0 = Model(inputs = embedding_model.input, outputs = embedding_model.layers[-2].output) # for embedding model variants
embedding_model_0.summary()

trainval_vec = embedding_model_0.predict(trainval_input1)
print(trainval_vec, trainval_vec.shape)


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1, 1, 2048)]      0         
                                                                 
 layer1 (Conv2D)             (None, 1, 1, 2048)        4196352   
                                                                 
 layer2 (Conv2D)             (None, 1, 1, 1024)        2098176   
                                                                 
Total params: 6,294,528
Trainable params: 6,294,528
Non-trainable params: 0
_________________________________________________________________
221/221 [==============================] - 0s 748us/step
[[[[0.96928316 0.         1.6717384  ... 0.         0.
    0.9863841 ]]]


 [[[1.4740046  0.         2.089198   ... 0.         0.
    1.1985756 ]]]


 [[[1.2754202  0.         1.7703605  ... 0.         0.
    0.1944209 ]]]


 ...


 [[[1.4740258  0.       

In [16]:
inputt = Input(shape = (1, 1, trainval_attributes.shape[1]))
hidden1 = Conv2DTranspose(1024, 1, name="layer1", activation='relu')(inputt)

embedding_model_1 = Model(inputs = inputt, outputs = hidden1)
embedding_model_1.summary()

w_list = Model(inputs = embedding_model_0.layers[-2].output, outputs = embedding_model_0.output).get_weights()
#print(weights_list2)

embedding_model_1.set_weights([np.transpose(w_list[0], (0, 1, 3, 2)), w_list[1]])

trainval_attributes = embedding_model_1.predict(trainval_input2)
print(trainval_attributes, trainval_attributes.shape)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1, 1, 2048)]      0         
                                                                 
 layer1 (Conv2DTranspose)    (None, 1, 1, 1024)        2098176   
                                                                 
Total params: 2,098,176
Trainable params: 2,098,176
Non-trainable params: 0
_________________________________________________________________
221/221 [==============================] - 0s 659us/step
[[[[0.         0.         0.02944161 ... 0.00646748 0.028888
    0.        ]]]


 [[[0.02042671 0.         0.04095376 ... 0.         0.04360095
    0.        ]]]


 [[[0.         0.         0.01596911 ... 0.00080441 0.03779721
    0.        ]]]


 ...


 [[[0.         0.         0.03696621 ... 0.01328099 0.02166263
    0.        ]]]


 [[[0.02109868 0.         0.03102086 ... 0.        

In [17]:

attribute_new = np.reshape(attribute_new, [attribute_new.shape[0], 1, 1, attribute_new.shape[1]])

# as labels range from 1 to 200, we have subtract 1
attribute_2 = embedding_model_1.predict(attribute_new)
print(attribute_2, attribute_2.shape)


7/7 [==============================] - 0s 997us/step
[[[[0.01001195 0.00122673 0.         ... 0.00130253 0.06237516
    0.        ]]]


 [[[0.         0.         0.01138241 ... 0.03856681 0.05814604
    0.        ]]]


 [[[0.02133051 0.         0.04888634 ... 0.01241675 0.05244125
    0.        ]]]


 ...


 [[[0.02042671 0.         0.04095376 ... 0.         0.04360095
    0.        ]]]


 [[[0.         0.         0.04200585 ... 0.         0.03273274
    0.        ]]]


 [[[0.02872183 0.         0.03969611 ... 0.         0.07809225
    0.01184638]]]] (200, 1, 1, 1024)


In [18]:


signature_2 = attribute_2.transpose()
signature_2 = np.reshape(signature_2, [signature_2.shape[0], signature_2.shape[3]])
print(signature_2, signature_2.shape)#(1024, 200)

trainval_sig = signature_2[:, (unique_labels_trainval)-1]
print("Signature for trainval:", trainval_sig.shape)# (1024, 150)


[[0.01001195 0.         0.02133051 ... 0.02042671 0.         0.02872183]
 [0.00122673 0.         0.         ... 0.         0.         0.        ]
 [0.         0.01138241 0.04888634 ... 0.04095376 0.04200585 0.03969611]
 ...
 [0.00130253 0.03856681 0.01241675 ... 0.         0.         0.        ]
 [0.06237516 0.05814604 0.05244125 ... 0.04360095 0.03273274 0.07809225]
 [0.         0.         0.         ... 0.         0.         0.01184638]] (1024, 200)
Signature for trainval: (1024, 150)


In [19]:
# by doing this modification, we are changing the range of trainval and test seen labels from 0 to 149 
# and test unseen labels from 0 to 49

k = 0
new_labels_trainval = np.zeros((len(labels_trainval), 1), dtype = 'int')
for labels in unique_labels_trainval:
    new_labels_trainval[labels_trainval == labels] = k
    k = k+1
    
print(new_labels_trainval, new_labels_trainval.shape)#(7057, 1)
print(np.unique(new_labels_trainval), np.unique(new_labels_trainval).shape)


[[146]
 [147]
 [ 26]
 ...
 [ 55]
 [115]
 [ 18]] (7057, 1)
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149] (150,)


In [20]:
#params for trainval and test set
m_trainval = new_labels_trainval.shape[0]# number of instances in training set: 7057
print(m_trainval)

z_trainval = len(unique_labels_trainval)# number of classes in training set: 150
print(z_trainval)


7057
150


In [21]:
from tensorflow.keras.utils import to_categorical
gt_trainval = to_categorical(new_labels_trainval, z_trainval)

print(gt_trainval, gt_trainval.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (7057, 150)


In [22]:
input1_shape = trainval_vec.shape[3]
print(input1_shape)

attribute_shape = trainval_sig.shape[0]
print(attribute_shape)

output_shape = z_trainval
print(output_shape)


1024
1024
150


In [23]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from keras.optimizers import SGD, Adam, Adagrad

# define model2 for attribute to class label mapping

input2 = Input(shape = (1, 1, attribute_shape))
flat = Flatten()(input2)
output = Dense(output_shape, name="output", activation='softmax')(flat)

model2 = Model(inputs = input2, outputs = output)


opt = Adam(learning_rate = 1e-2, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0001)
#opt = SGD(learning_rate = 1e-2, decay = 1e-6, momentum = 0.9, nesterov = True)
model2.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

model2.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1, 1, 1024)]      0         
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 output (Dense)              (None, 150)               153750    
                                                                 
Total params: 153,750
Trainable params: 153,750
Non-trainable params: 0
_________________________________________________________________


In [24]:
# define model1 for visual feature to class label mapping

input1 = Input(shape = (1,1, input1_shape))
#inter_pre = Dense(512, name="intermediate_previous", activation='relu')(input1)
inter = Conv1D(attribute_shape, kernel_size = 1, name = "intermediate", activation = 'linear')(input1)
flat = Flatten()(inter)
output = Dense(output_shape, name="output", activation='softmax')(flat)

model1 = Model(inputs = input1, outputs = output)

opt = Adam(learning_rate = 1e-2, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0001)
#opt = SGD(learning_rate = 1e-2, decay = 1e-6, momentum = 0.9, nesterov = True)
model1.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

model1.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1, 1, 1024)]      0         
                                                                 
 intermediate (Conv1D)       (None, 1, 1, 1024)        1049600   
                                                                 
 flatten_1 (Flatten)         (None, 1024)              0         
                                                                 
 output (Dense)              (None, 150)               153750    
                                                                 
Total params: 1,203,350
Trainable params: 1,203,350
Non-trainable params: 0
_________________________________________________________________


In [25]:
trainval_input1 = trainval_vec
print(trainval_input1.shape)

trainval_input2 = trainval_attributes
print(trainval_input2.shape)

trainval_output = gt_trainval
print(trainval_output.shape)

(7057, 1, 1, 1024)
(7057, 1, 1, 1024)
(7057, 150)


In [26]:
from tensorflow.keras.utils import Sequence
class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y
    
batch_size = 8
from sklearn.model_selection import train_test_split    


X_train1, X_val1, y_train1, y_val1 = train_test_split(trainval_input1, trainval_output, test_size = 0.2, random_state = 42)

#train_gen1 = DataGenerator(X_train1, y_train1, batch_size)   
#val_gen1 = DataGenerator(X_val1, y_val1, batch_size)


X_train2, X_val2, y_train2, y_val2 = train_test_split(trainval_input2, trainval_output, test_size = 0.2, random_state = 42)

#train_gen2 = DataGenerator(X_train2, y_train2, batch_size)   
#val_gen2 = DataGenerator(X_val2, y_val2, batch_size)

In [27]:
labels = res101['labels']# direct class labels
print('labels', labels, labels.shape)# 7057 x 1

test_seen_loc = np.squeeze(att_splits['test_seen_loc']-1)
print(np.unique(test_seen_loc), np.max(np.unique(test_seen_loc))) # smallest location: 0, largest location 11727

labels_test_seen = labels[test_seen_loc]
print('labels_test_seen', labels_test_seen, labels_test_seen.shape)

unique_labels_test_seen = np.unique(labels_test_seen) # labels min:1 max:200
print('unique_labels_test_seen', unique_labels_test_seen, unique_labels_test_seen.shape)# 150 classes

test_seen_vec = X_features[:, test_seen_loc].transpose()
print("Features for test seen:", test_seen_vec.shape)# (1764, 2048)

test_seen_vec = np.reshape(test_seen_vec, [test_seen_vec.shape[0], 1, 1, test_seen_vec.shape[1]])
test_seen_vec = embedding_model_0.predict(test_seen_vec)
print(test_seen_vec.shape)


test_seen_attributes = np.zeros((len(test_seen_loc), attribute_2.shape[1], attribute_2.shape[2], attribute_2.shape[3]))
for i in range(len(test_seen_loc)):
    test_seen_attributes[i] = attribute_2[int(labels_test_seen[i])-1]

print(test_seen_attributes, test_seen_attributes.shape)# 1764, 1, 1, 1024



test_seen_sig = signature_2[:, (unique_labels_test_seen)-1]
print("Signature for test seen:", test_seen_sig.shape)

labels [[151]
 [151]
 [151]
 ...
 [150]
 [150]
 [150]] (11788, 1)
[    0     3     6 ... 11716 11717 11727] 11727
labels_test_seen [[113]
 [107]
 [ 20]
 ...
 [136]
 [ 94]
 [180]] (1764, 1)
unique_labels_test_seen [  1   2   3   4   5   6   8   9  10  11  12  13  14  15  16  17  18  20
  22  23  24  25  26  27  28  30  31  32  33  35  37  38  39  40  41  42
  43  44  45  46  47  48  49  51  52  53  54  55  57  58  59  60  61  63
  64  65  66  67  70  71  73  74  75  76  77  78  81  82  83  84  85  86
  89  90  92  93  94  96  97  99 101 102 103 105 106 107 109 110 111 112
 113 114 115 117 118 119 121 123 126 127 128 130 131 132 133 134 135 136
 137 138 140 143 144 145 146 147 148 149 151 153 154 155 156 158 161 162
 163 164 165 168 169 170 172 173 175 177 178 180 181 183 184 186 188 190
 194 196 197 198 199 200] (150,)
Features for test seen: (1764, 2048)
56/56 [==============================] - 0s 753us/step
(1764, 1, 1, 1024)
[[[[0.         0.         0.00989528 ... 0.         0.05553

In [28]:

test_unseen_loc = np.squeeze(att_splits['test_unseen_loc']-1)
print(np.unique(test_unseen_loc), np.max(np.unique(test_unseen_loc))) # smallest location: 1046, largest location 35290

labels_test_unseen = labels[test_unseen_loc]
print('labels_test_unseen', labels_test_unseen, labels_test_unseen.shape)

unique_labels_test_unseen = np.unique(labels_test_unseen) # labels min:7 max:50
print('unique_labels_test_unseen', unique_labels_test_unseen, unique_labels_test_unseen.shape)# 10 classes


test_unseen_vec = X_features[:, test_unseen_loc].transpose()
print("Features for test unseen:", test_unseen_vec.shape) #(7913, 2048)


test_unseen_vec = np.reshape(test_unseen_vec, [test_unseen_vec.shape[0], 1, 1, test_unseen_vec.shape[1]])
test_unseen_vec = embedding_model_0.predict(test_unseen_vec)
print(test_unseen_vec.shape)

#test_unseen_attributes = np.zeros((len(test_unseen_loc), attribute_new.shape[1]))
#for i in range(len(test_unseen_loc)):
#    test_unseen_attributes[i] = attribute_new[int(labels_test_unseen[i])-1]

#print(test_unseen_attributes, test_unseen_attributes.shape)# (7913, 85)


#test_unseen_attributes_2 = model0.predict(test_unseen_attributes)
#print(test_unseen_attributes_2, test_unseen_attributes_2.shape)

test_unseen_sig = signature_2[:, (unique_labels_test_unseen)-1]

print("Signature for test unseen:", test_unseen_sig.shape)

[  178   179   180 ... 11785 11786 11787] 11787
labels_test_unseen [[152]
 [152]
 [152]
 ...
 [150]
 [150]
 [150]] (2967, 1)
unique_labels_test_unseen [  7  19  21  29  34  36  50  56  62  68  69  72  79  80  87  88  91  95
  98 100 104 108 116 120 122 124 125 129 139 141 142 150 152 157 159 160
 166 167 171 174 176 179 182 185 187 189 191 192 193 195] (50,)
Features for test unseen: (2967, 2048)
93/93 [==============================] - 0s 759us/step
(2967, 1, 1, 1024)
Signature for test unseen: (1024, 50)


In [29]:

l = 0
new_labels_test_seen = np.zeros((len(labels_test_seen), 1), dtype = 'int')
for labels in unique_labels_test_seen:
    new_labels_test_seen[labels_test_seen == labels] = l
    l = l+1
    
print(new_labels_test_seen, new_labels_test_seen.shape)# (5882, 1)



n_test_seen = new_labels_test_seen.shape[0]# 5882
print(n_test_seen)

z1_test_seen = len(unique_labels_test_seen)# 40
print(z1_test_seen)


m = 0
new_labels_test_unseen = np.zeros((len(labels_test_unseen), 1), dtype = 'int')
for labels in unique_labels_test_unseen:
    new_labels_test_unseen[labels_test_unseen == labels] = m
    m = m+1  

print(new_labels_test_unseen, new_labels_test_unseen.shape) #  (7913, 1)  

n_test_unseen = new_labels_test_unseen.shape[0]# 7913
print(n_test_unseen)

z1_test_unseen = len(unique_labels_test_unseen)# 10
print(z1_test_unseen)


[[ 90]
 [ 85]
 [ 17]
 ...
 [107]
 [ 76]
 [137]] (1764, 1)
1764
150
[[32]
 [32]
 [32]
 ...
 [31]
 [31]
 [31]] (2967, 1)
2967
50


In [43]:
iteration = 5
epochs1 = 100
epochs2 = 200

best_performance_micro = [0, 0, 0]
best_performance_macro = [0, 0, 0]

save_path = 'C:/Users/Admin/Sushree_Codes/Sush_3/Results/CUB/BertAttribute/with_LE/'
name = 'Remodel1_conv_CUB_LE_BertattT'
    
for i in range(iteration):
    X_train2_it = X_train2[(len(X_train2)//iteration)*i:(len(X_train2)//iteration)*(i+1)]
    X_val2_it = X_val2[(len(X_val2)//iteration)*i:(len(X_val2)//iteration)*(i+1)]
    y_train2_it = y_train2[(len(y_train2)//iteration)*i:(len(y_train2)//iteration)*(i+1)]
    y_val2_it = y_val2[(len(y_val2)//iteration)*i:(len(y_val2)//iteration)*(i+1)]
    
    train_gen2 = DataGenerator(X_train2_it, y_train2_it, batch_size)   
    val_gen2 = DataGenerator(X_val2_it, y_val2_it, batch_size)

    train_summary2 = model2.fit(train_gen2, epochs = epochs2, verbose = 0, callbacks = None, validation_data = val_gen2, 
                              shuffle = True, steps_per_epoch = len(train_gen2)//batch_size, 
                              validation_steps = len(val_gen2)//batch_size)

    print("iteration:", i)
    print('model 2 is trained:', 'training acc:', train_summary2.history['accuracy'][-1], ',',  
          'training loss:', train_summary2.history['loss'][-1], ',', 
          'validation acc:', train_summary2.history['val_accuracy'][-1], ',',
          'validation_loss:', train_summary2.history['val_loss'][-1])

    weights_list2 = model2.get_weights()
    #print(weights_list2)

    model1.layers[-1].set_weights(weights_list2)

    X_train1_it = X_train1[(len(X_train1)//iteration)*i:(len(X_train1)//iteration)*(i+1)]
    X_val1_it = X_val1[(len(X_val1)//iteration)*i:(len(X_val1)//iteration)*(i+1)]
    y_train1_it = y_train1[(len(y_train1)//iteration)*i:(len(y_train1)//iteration)*(i+1)]
    y_val1_it = y_val1[(len(y_val1)//iteration)*i:(len(y_val1)//iteration)*(i+1)]
    
    train_gen1 = DataGenerator(X_train1_it, y_train1_it, batch_size)   
    val_gen1 = DataGenerator(X_val1_it, y_val1_it, batch_size)
    
    for layer in model1.layers[2:]:
        layer.trainable = True

    train_summary1 = model1.fit(train_gen1, epochs = epochs1, verbose = 0, callbacks = None, validation_data = val_gen1, 
                              shuffle = True, steps_per_epoch = len(train_gen1)//batch_size, 
                              validation_steps = len(val_gen1)//batch_size)
    
    #print("iteration:", i)
    print('model 1 is trained:', 'training acc:', train_summary1.history['accuracy'][-1], ',',  
          'training loss:', train_summary1.history['loss'][-1], ',', 
          'validation acc:', train_summary1.history['val_accuracy'][-1], ',',
         'validation_loss:', train_summary1.history['val_loss'][-1])
    
    weights_list1 = Model(inputs = model1.input, outputs = model1.layers[-1].output).get_weights()
    
    #predictions
    #outputs_seen = np.matmul(np.matmul(test_seen_vec, np.matmul(weights_list1[0], weights_list1[2])), test_seen_sig)
    #outputs_seen = np.matmul(np.matmul(test_seen_vec, weights_list1[0]), test_seen_sig)
    
    outputs_seen = np.matmul(np.matmul(test_seen_vec, weights_list1[0][0]), test_seen_sig)
    preds_seen = np.array([np.argmax(output) for output in outputs_seen])
    
    cm_seen = confusion_matrix(new_labels_test_seen, preds_seen)
    #print(cm)
    # Compute macro average (averaging performance metrics by first calculating the metric separately for each class and 
    # then averaging these class-specific metrics)
    cm_seen_micro = cm_seen.astype('float') / cm_seen.sum(axis=1)[:, np.newaxis]
    #print(cm)
    avg_seen_micro = (sum(cm_seen_micro.diagonal())/len(unique_labels_test_seen))*100

    avg_seen_macro = (sum(cm_seen.diagonal())/len(new_labels_test_seen))*100
    
    #predictions
    #outputs_unseen = np.matmul(np.matmul(test_unseen_vec, weights_list1[0]), test_unseen_sig)
    outputs_unseen = np.matmul(np.matmul(test_unseen_vec, weights_list1[0][0]), test_unseen_sig)
    
    preds_unseen = np.array([np.argmax(output) for output in outputs_unseen])
    
    cm_unseen = confusion_matrix(new_labels_test_unseen, preds_unseen)
    # Compute macro average (averaging performance metrics by first calculating the metric separately for each class and 
    # then averaging these class-specific metrics)
    cm_unseen_micro = cm_unseen.astype('float') / cm_unseen.sum(axis=1)[:, np.newaxis]
    avg_unseen_micro = (sum(cm_unseen_micro.diagonal())/len(unique_labels_test_unseen))*100

    avg_unseen_macro = (sum(cm_unseen.diagonal())/len(new_labels_test_unseen))*100
    
    harmonic_micro = (2*avg_seen_micro*avg_unseen_micro) / (avg_seen_micro + avg_unseen_micro)
    harmonic_macro = (2*avg_seen_macro*avg_unseen_macro) / (avg_seen_macro + avg_unseen_macro)
    
    print('micro average')
    print('seen accuracy:', avg_seen_micro, 'unseen accuracy:', avg_unseen_micro, 'harmonic mean:', harmonic_micro)
    
    print('macro average')
    print('seen accuracy:', avg_seen_macro, 'unseen accuracy:', avg_unseen_macro, 'harmonic mean:', harmonic_macro)
    
    if harmonic_micro > best_performance_micro[2]:
        best_performance_micro = [avg_seen_micro, avg_unseen_micro, harmonic_micro]
        model1.save_weights(save_path + 'bw_micro_' + name + '.h5', overwrite=True)
        
    if harmonic_macro > best_performance_macro[2]:
        best_performance_macro = [avg_seen_macro, avg_unseen_macro, harmonic_macro]
        model1.save_weights(save_path + 'bw_macro_' + name + '.h5', overwrite=True)
        
    print('best accuracy micro','seen accuracy:', best_performance_micro[0], 'unseen accuracy:', best_performance_micro[1], 'harmonic mean:', best_performance_micro[2])
    print('best accuracy macro', 'seen accuracy:', best_performance_macro[0], 'unseen accuracy:', best_performance_macro[1], 'harmonic mean:', best_performance_macro[2])
    
    print('-----------------------------------------------------------------------------------------------------------')
    weights_list3 = model1.get_weights()
    model2.set_weights(weights_list3[2:])

iteration: 0
model 2 is trained: training acc: 1.0 , training loss: 0.3417608439922333 , validation acc: 1.0 , validation_loss: 0.3107287883758545
model 1 is trained: training acc: 0.9852941036224365 , training loss: 1.1257739067077637 , validation acc: 0.53125 , validation_loss: 133.14451599121094
micro average
seen accuracy: 46.94961226137696 unseen accuracy: 43.26442234408404 harmonic mean: 45.03174838927424
macro average
seen accuracy: 46.03174603174603 unseen accuracy: 43.10751600943714 harmonic mean: 44.52166606649862
best accuracy micro seen accuracy: 46.94961226137696 unseen accuracy: 43.26442234408404 harmonic mean: 45.03174838927424
best accuracy macro seen accuracy: 46.03174603174603 unseen accuracy: 43.10751600943714 harmonic mean: 44.52166606649862
-----------------------------------------------------------------------------------------------------------
iteration: 1
model 2 is trained: training acc: 0.9852941036224365 , training loss: 0.36965346336364746 , validation acc:

In [44]:
gt_test_seen = to_categorical(new_labels_test_seen, z1_test_seen)

print(gt_test_seen, gt_test_seen.shape)

#test_seen_vec = np.reshape(test_seen_vec, [test_seen_vec.shape[0], 1, 1, test_seen_vec.shape[1]])
res1 = model1.evaluate(test_seen_vec, gt_test_seen)

p1 = model1.predict(test_seen_vec, verbose = 0)

import tensorflow
cce = tensorflow.keras.losses.CategoricalCrossentropy()
print('cce = ', cce(gt_test_seen, p1).numpy())

#test_seen_attributes = np.reshape(train_attributes, [test_seen_attributes.shape[0], 1, train_attributes.shape[1]])
#test_seen_attributes = np.reshape(test_seen_attributes, [test_seen_attributes.shape[0], 1, 1, test_seen_attributes.shape[1]])

res2 = model2.evaluate(test_seen_attributes, gt_test_seen)

p2 = model2.predict(test_seen_attributes, verbose = 0)

import tensorflow
cce = tensorflow.keras.losses.CategoricalCrossentropy()
print('cce = ', cce(gt_test_seen, p2).numpy())


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (1764, 150)
56/56 [==============================] - 0s 2ms/step - loss: 169.2656 - accuracy: 0.5805
cce =  6.6388063
56/56 [==============================] - 0s 2ms/step - loss: 0.3626 - accuracy: 0.9932
cce =  0.36259893


In [45]:

accuracy_seen_updated = res1[1]*100
unseen_accuracy = 43.26
h = (2*accuracy_seen_updated*unseen_accuracy) / (accuracy_seen_updated + unseen_accuracy)
print(h)


accuracy_seen_updated2 = ((res1[1]*100)+(res2[1]*100))/2
print(accuracy_seen_updated2)
h = (2*accuracy_seen_updated2*unseen_accuracy) / (accuracy_seen_updated2 + unseen_accuracy)
print(h)

49.57538091726869
78.68480682373047
55.82697339649531


In [46]:
from sklearn.metrics import precision_recall_fscore_support

pp1 = np.array([np.argmax(output) for output in p1])
pp2 = np.array([np.argmax(output) for output in p2])

seen_macro1 = precision_recall_fscore_support(new_labels_test_seen, pp1, average = 'macro')
seen_macro2 = precision_recall_fscore_support(new_labels_test_seen, pp2, average = 'macro')
print('precision_seen_macro', (seen_macro1[0] + seen_macro2[0])/2, 'recall_seen_macro', (seen_macro1[1] + seen_macro2[1])/2, 'f1_seen_macro', (seen_macro1[2] + seen_macro2[2])/2)


seen_micro1 = precision_recall_fscore_support(new_labels_test_seen, pp1, average = 'micro')
seen_micro2 = precision_recall_fscore_support(new_labels_test_seen, pp2, average = 'micro')
print('precision_seen_micro', (seen_micro1[0] + seen_micro2[0])/2, 'recall_seen_micro', (seen_micro1[1] + seen_micro2[1])/2, 'f1_seen_micro', (seen_micro1[2] + seen_micro2[2])/2)

unseen_macro = precision_recall_fscore_support(new_labels_test_unseen, preds_unseen, average = 'macro')
unseen_micro = precision_recall_fscore_support(new_labels_test_unseen, preds_unseen, average = 'micro')

print('precision_unseen_macro', unseen_macro[0], 'recall_unseen_macro', unseen_macro[1], 'f1_unseen_macro', unseen_macro[2])
print('precision_unseen_micro', unseen_micro[0], 'recall_unseen_micro', unseen_micro[1], 'f1_unseen_micro', unseen_micro[2])


precision_seen_macro 0.8107142469382072 recall_seen_macro 0.7941405568832038 f1_seen_macro 0.78318607639317
precision_seen_micro 0.7868480725623583 recall_seen_micro 0.7868480725623583 f1_seen_micro 0.7868480725623583
precision_unseen_macro 0.4734833441734912 recall_unseen_macro 0.4229350651116947 f1_unseen_macro 0.37675080224968105
precision_unseen_micro 0.4213009774182676 recall_unseen_micro 0.4213009774182676 f1_unseen_micro 0.4213009774182676


C:\Users\Admin\anaconda3\envs\en3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
